In [ ]:
from typing import Dict
import torch
import os,sys
import itertools
from collections import defaultdict
from sklearn import metrics as mtrcs
from collections import Counter
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import random
import string
import networkx as nx
from tqdm import tqdm
from plotly import graph_objs as go
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.model_selection import ParameterGrid
import seaborn as sns
import sys
sys.path.append('../')
RANDOM_SEED = 1
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
import pickle

def compute_ranks(scores, source_nodes, target_nodes, labels):
    """
    Given source nodes, target nodes, and prediction score, compute the rank associated with every positive triple

    :param predictionsDF: Dataframe of triplet, and prediction score
    :return: Computed ranks for each positive triple
    """

    ranks = {}
    source_nodes_new = []

    mask_neg = labels == "N"
    scores_neg = scores[mask_neg]
    sources_neg = source_nodes[mask_neg]
    irx = np.argsort(-scores_neg)
    scores_neg = scores_neg[irx]
    sources_neg = sources_neg[irx]

    scores_neg_dict = {}
    sources_neg_dict = {}
    for i, sn in tqdm(enumerate(sources_neg)):
        v = sources_neg_dict.get(sn, [])
        v.append(i)
        sources_neg_dict[sn] = v
    unique_sources = list(sources_neg_dict.keys())

    mask_pos = np.logical_not(mask_neg)
    scores_pos = scores[mask_pos]
    sources_pos = source_nodes[mask_pos]
    for i in tqdm(range(sum(mask_pos))):
        score = scores_pos[i]
        sourceNode = sources_pos[i]
        scores_negative = scores_neg[sources_neg_dict[sourceNode]]
        rank = np.sum(scores_negative > score) + 1

        # detect ties and put the rank in the middle of the tie
        irx = np.where(scores_negative == score)[0]
        if len(irx) > 1:
            rank = int((irx[0] + irx[-1]) / 2)

        if sourceNode not in source_nodes_new:
            ranks[sourceNode] = []
            source_nodes_new.append(sourceNode)
        ranks[sourceNode].append(rank)
    return ranks

def compute_precision_at_k(prediction_scores: pd.DataFrame, k: int) -> Dict[str, float]:
    """
    A function to compute precision@k from a dataframe with prediction scores containing scores for all triples.
    IMPORTANT: Assumes that training triples have been removed from prediction_scores upstream. Also assumes that
    scores for each gene for each disease are present in the dataframe (apart from scores for true training triples
    that have been removed).

    Precision@k is the fraction of true links in the test set in the top k predictions (when true links in the training
    set have been removed). Therefore, it's meant to act as a kind of proxy for the fraction of true and novel links
    we'd expect to see in the top k predictions for a disease.

    :param prediction_scores: dataframe containing scores for all triples
    :param k: k for which precision@k calculated
    :return: dictionary with precision@k for each disease
    """
    precision_per_disease = {}
    disease_ids = list(set(prediction_scores["source_node"]))

    for disease_id in tqdm(disease_ids):
        disease_id_scores = prediction_scores[
            prediction_scores["source_node"] == disease_id
        ]
        disease_id_scores_sorted = disease_id_scores.sort_values(
            by="score", ascending=False
        )
        top_k = disease_id_scores_sorted.iloc[:k]
        number_true = len(top_k[top_k["label"] == "P"])
        disease_precision = number_true / k
        precision_per_disease[disease_id] = disease_precision

    return precision_per_disease

def compute_precision_at_k_alt(
    prediction_scores: pd.DataFrame, k: int
) -> Dict[str, float]:
    """
    A function to compute precision@k from a dataframe with prediction scores containing scores for all triples.
    IMPORTANT: Assumes that training triples have been removed from prediction_scores upstream. Also assumes that
    scores for each gene for each disease are present in the dataframe (apart from scores for true training triples
    that have been removed).

    Precision@k is the fraction of true links in the test set in the top k predictions (when true links in the training
    set have been removed). Therefore, it's meant to act as a kind of proxy for the fraction of true and novel links
    we'd expect to see in the top k predictions for a disease.

    :param prediction_scores: dataframe containing scores for all triples
    :param k: k for which precision@k calculated
    :return: dictionary with precision@k for each disease
    """
    precision_per_disease = defaultdict(float)

    # Group by disease_id and sort within each group
    prediction_scores_sorted = prediction_scores.sort_values(
        by=["source_node", "score"], ascending=[True, False]
    )

    # Calculate precision for each disease_id
    for disease_id, group in prediction_scores_sorted.groupby("source_node"):
        top_k_group = group.head(k)
        number_true = len(top_k_group[top_k_group["label"] == "P"])
        disease_precision = number_true / k
        precision_per_disease[disease_id] = disease_precision

    return dict(precision_per_disease)

def compute_hit_at_k_source_ranks(source_ranks, k: int) -> float:
    """
    Compute hit@k score from a triplet, positive or negative lablel and predicted score

    :param k: parameter to be used for hit@k computation
    :return: Average hit@k across all positive triples
    """
    rank_average = []
    for _, value in source_ranks.items():
        hit_at_k_source = hit_at_k(value, k=k)
        rank_average.append(hit_at_k_source)
    # hit_at_k_score = np.round(float(sum(rank_average) / len(rank_average)), 3)
    hit_at_k_score = float(sum(rank_average) / len(rank_average))
    return hit_at_k_score

def hit_at_k(ranks: list, k: int) -> float:
    """
    hits@k describes the fraction of true entities that appear in the first k entities of the sorted rank list.

    :param ranks: Calculated ranks
    :param k: The maximum rank
    :return: hit@k score
    """
    ranks = np.asarray(ranks)
    try:
        htk = float(np.sum(ranks <= k) / len(ranks))
    except ZeroDivisionError:
        return 0
    return htk




In [ ]:
# Loading the prediction table
link_to_predict = 'gene_mf'
prediction_file = 'trained_models/'+link_to_predict+'/'+link_to_predict+'_predictions.tsv'
df_predictions = pd.read_csv(prediction_file,sep='\t',index_col=0)
display(df_predictions)

In [ ]:
# Prediction table for a particular gene
gene = 'NODAL'
#gene = 'OR2L13'
#gene = "SLC7A9"
display(df_predictions[df_predictions['GENE']==gene].sort_values("DotProduct",ascending=False).head(40))


In [ ]:
gene = 'CDX2'
display(df_predictions[df_predictions['GENE']==gene].sort_values("DotProduct",ascending=False).head(40))


In [ ]:
# Compute AUCs

tab = df_predictions.copy()

tab['Status'].replace('None', 0,inplace=True)
tab['Status'].replace('New', 0,inplace=True)
tab['Status'].replace(np.nan, 0,inplace=True)
tab['Status'].replace('Train', 1,inplace=True)
tab['Status'].replace('Test', 1,inplace=True)
tab['Status'].replace('Validation', 1,inplace=True)


feature = 'DotProduct'

preds = list(tab[feature])
labels = list(tab["Status"])

fpr, tpr, thresholds = mtrcs.roc_curve(labels, preds)

# Using J-statistic: https://en.wikipedia.org/wiki/Youden%27s_J_statistic
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print("Best Threshold=%f" % (best_thresh))

roc_auc = mtrcs.roc_auc_score(labels, preds)

plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.plot([0, 1], [0, 1], "r--")  # diagonal roc curve of a random classifier
plt.scatter(
    fpr[ix], tpr[ix], marker="o", color="black", label="Best=%0.2f" % best_thresh
)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.title(f"ROC curve for {feature}, "+feature)
plt.show()

del(tab)

In [ ]:
### Compute hit@k


feature = "DotProduct"

tab = df_predictions[['DISEASE','GENE',feature, "Status"]].copy()

tab['Status'].replace('None', 0,inplace=True)
tab['Status'].replace('New', 0,inplace=True)
tab['Status'].replace(np.nan, 0,inplace=True)
tab['Status'].replace('Train', 1,inplace=True)
tab['Status'].replace('Test', 1,inplace=True)
tab['Status'].replace('Validation', 1,inplace=True)

tab.index = range(len(tab))
print(np.sum(tab['Status']))

direction = 'disease_gene'
direction = 'gene_disease'

if direction=='disease_gene':
    tab.rename(
        columns={
            "DISEASE": "source_node",
            feature: "score",
            "GENE": "target_node",
            "Status": "label",
        },
        inplace=True,
    )
else:
    tab.rename(
        columns={
            "GENE": "source_node",
            feature: "score",
            "DISEASE": "target_node",
            "Status": "label",
        },
        inplace=True,
    )

tab['label'].replace(0, "N",inplace=True)
tab['label'].replace(1, "P",inplace=True)

#display(tab)

source_ranks = compute_ranks(np.array(tab['score']),
                            np.array(tab['source_node']),
                            np.array(tab['target_node']),
                            np.array(tab['label']))
k = 10
hit_at_k_score = compute_hit_at_k_source_ranks(source_ranks, k=k)
print(f"hit_at_{k}_score={hit_at_k_score}")
#print(f'precision@{k}={compute_precision_at_k(tab,k=k)}')
k = 100
hit_at_k_score = compute_hit_at_k_source_ranks(source_ranks, k=k)
print(f"hit_at_{k}_score={hit_at_k_score}")
k = 1000
hit_at_k_score = compute_hit_at_k_source_ranks(source_ranks, k=k)
print(f"hit_at_{k}_score={hit_at_k_score}")

In [ ]:
# Computing the precision@k
#tab.to_csv(root_folder+'/data/temp.tsv',sep='\t')
k = 10
precision_per_source = compute_precision_at_k_alt(tab,k)

dfgene2hugo = pd.read_csv('tests/geneid2hugo.tsv',sep='\t')
gene2hugo = {str(row['GeneID']):row['HUGO'] for i,row in dfgene2hugo.iterrows()}
hugo2geneid = {row['HUGO']:str(row['GeneID']) for i,row in dfgene2hugo.iterrows()}

zeroconnected = []
for disease_id, group in tqdm(tab.groupby("source_node")):
    number_true = len(group[group["label"] == "P"])
    if number_true<k:
        zeroconnected.append(disease_id)
print('Number of genes used to estimate precision:',len(precision_per_source)-len(zeroconnected))

zeroconnected = set(zeroconnected)
vals = [precision_per_source[k] for k in precision_per_source if not k in zeroconnected]
print(f'Precision@{k}',np.mean(vals))
plt.hist(vals,bins=50)
plt.show()